#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo
from datetime import datetime as dt

In [2]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [3]:
collection.delete_many({})

In [4]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [5]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Nettoyer les données

Tout d'abord, on convertit les types des variables que nous jugeons nécessaires. 
* deadline : datetime
* launched : datetime
* goal : float
* pledged : float

Lors du changement de class de la colonne goal, j'ai remarqué que cette colonne possédait une date à la ligne 66 141. Je l'ai alors supprimé.
Ensuite, il faut enlever les lignes dupliquées et les valeurs manquantes.

Enfin, j'ai rennomé la colonne ID en _id pour qu'elle soit plus adapté au traitement de données Mongo.

In [6]:
df_ks.shape

(150000, 14)

In [7]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


In [8]:
df_ks['deadline'] = pd.to_datetime(df_ks['deadline'], errors = 'coerce')
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors = 'coerce')

In [9]:
df_ks[df_ks['goal'] == '2014-04-17']

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
66141,85964225,Debut Album from Michael Adam,Grace is Leaving,Indie Rock,Music,NaT,2014-04-17,NaT,2014-04-02 21:56:35,850.0,successful,32,US,850.0


In [10]:
df_ks.drop(66141, inplace = True)

In [11]:
df_ks['goal'] = df_ks['goal'].astype('float')
df_ks['pledged'] = df_ks['pledged'].astype('float')

In [12]:
df_ks.duplicated().sum()

0

In [13]:
df_ks.isna().sum().sum()

1484

In [14]:
df_ks_clean = df_ks.dropna()

In [15]:
df_ks_clean.shape

(148515, 14)

In [16]:
df_ks_clean = df_ks_clean.rename(columns = {'ID' : '_id'})

In [17]:
df_ks_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148515 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   _id               148515 non-null  int64         
 1   name              148515 non-null  object        
 2   category          148515 non-null  object        
 3   main_category     148515 non-null  object        
 4   currency          148515 non-null  object        
 5   deadline          148515 non-null  datetime64[ns]
 6   goal              148515 non-null  float64       
 7   launched          148515 non-null  datetime64[ns]
 8   pledged           148515 non-null  float64       
 9   state             148515 non-null  object        
 10  backers           148515 non-null  object        
 11  country           148515 non-null  object        
 12  usd pledged       148515 non-null  object        
 13  usd_pledged_real  148515 non-null  float64       
dtypes: d

### Importer les données

Il faut convertir la base de données en une liste de dictionnaires. Chaque ligne de df_ks_clean sera représenté en dictionnaire.

In [18]:
DOCUMENTS = df_ks_clean.to_dict(orient = 'records')

In [ ]:
DOCUMENTS

In [19]:
collection.insert_many(DOCUMENTS)

## Question 1  

#### Récupérer les 5 projets ayant reçu le plus de promesse de dons.

On récupère les documents stockés dans la collection, puis on les trie dans un ordre décroissant selon la promesse de don  et on limite aux 5 premiers documents. On obtient alors les 5 documents ayant le usd_pledged_real les plus élevé.

In [20]:
cur = collection.find().sort([("usd_pledged_real", -1)]).limit(5)

for document in cur :
    print('-----')
    print(document)

-----
{'_id': 342886736, 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': datetime.datetime(2014, 8, 30, 0, 0), 'goal': 50000.0, 'launched': datetime.datetime(2014, 7, 8, 10, 14, 37), 'pledged': 13285226.36, 'state': 'successful', 'backers': '62642', 'country': 'US', 'usd pledged': '13285226.36', 'usd_pledged_real': 13285226.36}
-----
{'_id': 2103598555, 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': datetime.datetime(2016, 6, 30, 0, 0), 'goal': 1000000.0, 'launched': datetime.datetime(2016, 5, 24, 15, 49, 52), 'pledged': 12779843.49, 'state': 'successful', 'backers': '66673', 'country': 'US', 'usd pledged': '12779843.49', 'usd_pledged_real': 12779843.49}
-----
{'_id': 1033978702, 'name': 'OUYA: A New Kind of Video Game Console', 'category': 'Gaming Hardware', 'main_category': 'Games', 'cu

## Question 2

#### Compter le nombre de projets ayant atteint leur but.

Les projets ayant atteint leur but sont ceux à l'état réussite, c'est à dire state = successful. Nous comptons donc tous les documents avec cette valeur.

In [21]:
cur = collection.count_documents({"state" : "successful"})
cur

52998

Il y a 52 998 projets qui ont atteint leur but.

## Question 3

#### Compter le nombre de projets pour chaque catégorie.

On regroupe les projets par catégories puis on incrémente avec $sum$ qui fait la somme de 1 pour chaque chaque projet. La fonction $aggregate$ me permet de faire cela en regroupant les documents par catégories et d'effectuer l'opération de somme sur chaque catégorie.

In [23]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "Nombre de projets" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Rock', 'Nombre de projets': 2707},
 {'_id': 'Mobile Games', 'Nombre de projets': 650},
 {'_id': 'Film & Video', 'Nombre de projets': 3657},
 {'_id': 'Fiction', 'Nombre de projets': 3703},
 {'_id': 'Performance Art', 'Nombre de projets': 855},
 {'_id': 'Comics', 'Nombre de projets': 1931},
 {'_id': 'Periodicals', 'Nombre de projets': 517},
 {'_id': 'Fantasy', 'Nombre de projets': 132},
 {'_id': 'Animation', 'Nombre de projets': 1017},
 {'_id': 'Festivals', 'Nombre de projets': 313},
 {'_id': 'Print', 'Nombre de projets': 287},
 {'_id': 'Live Games', 'Nombre de projets': 394},
 {'_id': 'Documentary', 'Nombre de projets': 6497},
 {'_id': 'Graphic Novels', 'Nombre de projets': 702},
 {'_id': 'Ceramics', 'Nombre de projets': 128},
 {'_id': 'Robots', 'Nombre de projets': 237},
 {'_id': 'Conceptual Art', 'Nombre de projets': 393},
 {'_id': 'Quilts', 'Nombre de projets': 32},
 {'_id': "Farmer's Markets", 'Nombre de projets': 175},
 {'_id': 'Family', 'Nombre de projets': 130},
 {'_id'

Il y a beaucoup de catégories distinctes, alors j'ai réitéré le comptage pour les catégories principales.

In [24]:
cur = collection.aggregate([{"$group" : {"_id" : "$main_category", "Nombre de projets" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Music', 'Nombre de projets': 19657},
 {'_id': 'Crafts', 'Nombre de projets': 3463},
 {'_id': 'Technology', 'Nombre de projets': 12871},
 {'_id': 'Journalism', 'Nombre de projets': 1807},
 {'_id': 'Photography', 'Nombre de projets': 4176},
 {'_id': 'Theater', 'Nombre de projets': 4305},
 {'_id': 'Publishing', 'Nombre de projets': 15721},
 {'_id': 'Games', 'Nombre de projets': 13987},
 {'_id': 'Art', 'Nombre de projets': 11260},
 {'_id': 'Food', 'Nombre de projets': 9805},
 {'_id': 'Comics', 'Nombre de projets': 4139},
 {'_id': 'Dance', 'Nombre de projets': 1488},
 {'_id': 'Fashion', 'Nombre de projets': 8943},
 {'_id': 'Film & Video', 'Nombre de projets': 24982},
 {'_id': 'Design', 'Nombre de projets': 11911}]

## Question 4

#### Compter le nombre de projets français ayant été instanciés avant 2016.

On va chercher les projets français ($country$ = $FR$) dont la date de $launched$ précède $2016$ avec l'utilisation de l'opérateur *plus petit* $lt$. De plus, les deux conditions sont connectées par l'opérateur logique $and$.

*len(list(cur)) est équivalent à l'utilisation de la fonction count_documents()*

In [25]:
cur = collection.find({"$and" : [{"country" : "FR"}, {"launched" : {"$lt" : dt.strptime("2016-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")}}]})
len(list(cur))

330

Il y a 330 projets KickStarter français qui ont été lancé avant 2016.

## Question 5

#### Récupérer les projets américains ayant demandé plus de 200 000 dollars.

On va cherché les projets de $country$ = $US$ et qui ont un $goal$ supérieur $gt$ à $200 000$. On va limité l'affichage à 5 projets afin d'effectuer une vérification sur ceux-là.

In [26]:
cur = collection.find({"$and" : [{"country" : "US"}, {"goal" : {"$gt" : 200000.0}}]})
list(cur.limit(5))

[{'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': datetime.datetime(2012, 9, 14, 0, 0),
  'goal': 287000.0,
  'launched': datetime.datetime(2012, 8, 13, 23, 14, 2),
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': datetime.datetime(2012, 6, 1, 0, 0),
  'goal': 250000.0,
  'launched': datetime.datetime(2012, 5, 1, 20, 49, 58),
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'currency': 'USD',
  'deadline': datetime.datetime(2013, 11, 4, 0, 

## Question 6 

#### Compter le nombre de projet ayant "Sport" dans leur nom

In [27]:
collection.create_index([("name",  "text")])

'name_text'

In [28]:
cur = collection.count_documents( {"$text": {"$search": "Sport"}} )
cur

316

Il y a 316 projets avec le mot "Sport" dans son nom.